In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

In [ ]:
def load_dataset():
    phishing_emails = [
        "Your account has been compromised. Click here to reset your password immediately.",
        "You've won a free iPhone! Visit this link to claim your prize.",
        "Update your billing information to avoid service suspension.",
        "We noticed a login attempt from an unknown device. Verify your identity now.",
        "Urgent: Your account will be locked in 24 hours unless you respond."
    ]